# Lab2 - PuLP Library

<b>Information on group members:</b><br>
1) 150284, Sofya Aksenyuk <br>
2) 150281, Uladzimir Ivashka

In [3]:
from pulp import *  
import numpy as np
import pandas as pd

## Introduction - brief tutorial on PuLP

In [4]:
# Create an LpProblem instance; LpMaximize = objective function is to be maximized
model = LpProblem(name="some-problem", sense=LpMaximize)

In [5]:
# Initialize the decision variables. We can set the name and lower bound as well.
# To create an array of variables, you can use comprehensions of LpProblem.dicts.

x1 = LpVariable(name="x1", lowBound=0)
x2 = LpVariable(name="x2", lowBound=0)

In [6]:
# An example expression
expression = 3 * x1 + 2 * x2
type(expression)

pulp.pulp.LpAffineExpression

In [7]:
# An example constraint
constraint = 2 * x1 + 3 * x2 >= 5
type(constraint)

pulp.pulp.LpConstraint

Ok, let's now use PuLP to solve the below problem:
$max$ $4x_1 + 2x_2$ <br>
s.t.<br>
     $1x_1 + 1x_2 \geq 1$ <br>
     $2x_1 + 1x_2 \leq 6$ <br>
     $-1x_1 + x_2 = 1$ <br>
     $x_1 \geq 0$ <br>
     $x_2 \geq 0$ 

In [8]:
# Problem
model = LpProblem(name="test-problem", sense=LpMaximize)

x1 = LpVariable(name="x1", lowBound=0)
x2 = LpVariable(name="x2", lowBound=0)

model += (1 * x1 + 1*x2 >= 1, "#1 constraint")
model += (2 * x1 + 1*x2 <= 6, "#2 constraint")
model += (-1 * x1 + 1*x2 == 1, "#3 constraint")

# Objective function
obj_func = 4*x1 + 2 * x2
model += obj_func

model

test-problem:
MAXIMIZE
4*x1 + 2*x2 + 0
SUBJECT TO
#1_constraint: x1 + x2 >= 1

#2_constraint: 2 x1 + x2 <= 6

#3_constraint: - x1 + x2 = 1

VARIABLES
x1 Continuous
x2 Continuous

In [9]:
# Solve the problem
status = model.solve()

In [10]:
# Print status
print(f"status: {model.status}, {LpStatus[model.status]}")

status: 1, Optimal


In [11]:
# Print objective value
print(f"objective: {model.objective.value()}")

objective: 12.000000199999999


In [12]:
# Print constraint values
for name, constraint in model.constraints.items():  print(f"{name}: {constraint.value()}")

#1_constraint: 3.3333334
#2_constraint: 9.999999983634211e-08
#3_constraint: 0.0


In [13]:
model.variables()

[x1, x2]

In [14]:
print(x1.value())
print(x2.value())

1.6666667
2.6666667


### The same code but using dictionaries and other nice tricks


In [15]:
model = LpProblem(name="some-problem", sense=LpMaximize)

In [16]:
var_names = ["First", "Second"]
x = LpVariable.dicts("x", var_names, 0)
x

{'First': x_First, 'Second': x_Second}

In [17]:
const_names = ["GE", "LE", "EQ"]
sense = [1, -1, 0] # GE, LE, EQ
coefs = [[1,1],[2,1],[-1,1]] # Matrix coefs
rhs = [1, 6, 1] 

for c, s, r, cn in zip(coefs, sense, rhs, const_names):
    expr = lpSum([x[var_names[i]] * c[i] for i in range(2)])
    model += LpConstraint(e=expr, sense = s, name = cn, rhs = r)
    
obj_coefs = [4,2]
model += lpSum([x[var_names[i]] * obj_coefs[i] for i in range(2)])
    
model

some-problem:
MAXIMIZE
4*x_First + 2*x_Second + 0
SUBJECT TO
GE: x_First + x_Second >= 1

LE: 2 x_First + x_Second <= 6

EQ: - x_First + x_Second = 1

VARIABLES
x_First Continuous
x_Second Continuous

In [18]:
status = model.solve()
print(f"status: {model.status}, {LpStatus[model.status]}")
print(f"objective: {model.objective.value()}")
for n in var_names: print(x[n].value())

status: 1, Optimal
objective: 12.000000199999999
1.6666667
2.6666667


# Homework - use the PuLP library to solve the following OR problem 

Johnson & Sons company manufactures 6 types of toys. Each toy is produced by utilizing at least one machine. Each toy takes some production time on each Machine 1-4 (see Table below). For instance, product A requires 4 minutes on Machine 1, 4 minutes on Machine 2, 0 Minutes on Machine 3, and 0 minutes on Machine 4 (all machines must be utilized to produce a toy unless the production time equals 0). Each machine is available for a different number of hours per week. The company's goal is to identify the number (product-mix) of toys that have to be manufactured in order to maximize the income (can be continuous). Notably, each toy can be sold for a different price. Due to market expectations, the company wants to manufacture twice as many F toys as A. Furthermore, the number of toys B should equal C. Solve this problem using the PuLP library. Prepare a report containing information on:
<ol>
<li>The number of toys to manufacture;</li>
<li>The expected income;</li>
<li>The production time required on each machine;</li>
</ol>
Consider the total value and partial values, i.e., considered for each toy A-F separately (e.g., income resulted by selling toy A). Also, answer the following questions concerning the found solution: 
<ol>
<li>Which toy(s) production should be focused on?  </li>
<li>Is there a toy that can be excluded for consideration for production? </li>
<li>Is there a machine that is not fully utilized?</li>
</ol>

| Toy | Machine 1 | Machine 2 | Machine 3 | Machine 4 | Selling price |
| --- | --- | --- | --- | --- | --- |
| A | 4 (minutes!) | 4 | 0 | 0 | 2.50 USD |
| B | 0 | 3 | 3 | 0 | 1.00 USD |
| C | 5 | 0 | 2 | 5 | 4.00 USD |
| D | 2 | 4 | 0 | 4 | 3.00 USD |
| E | 0 | 4 | 5 | 2 | 3.50 USD |
| F | 2 | 2 | 1 | 1 | 4.00 USD |
| Production time limit (hours!) | 120 | 60 |  80 |  120 |  |

Max Z = 2.5A + 1B + 4C + 3D + 3.5E + 4F

Constraints:

(1) 4A + __ + 5C + 2D + __ + 2F <= 7200

(2) 4A + 3B + __ + 4D + 4E + 2F <= 3600

(3) __ + 3B + 2C + __ + 5E + 1F <= 4800

(4) __ + __ + 5C + 4D + 2E + 1F <= 7200

(5) B - C = 0

(6) F - 2A = 0

A, B, C, D, E, F >= 0

In [45]:
model = LpProblem(name="test-problem", sense=LpMaximize)

a = LpVariable(name="A", lowBound=0)
b = LpVariable(name="B", lowBound=0)
c = LpVariable(name="C", lowBound=0)
d = LpVariable(name="D", lowBound=0)
e = LpVariable(name="E", lowBound=0)
f = LpVariable(name="F", lowBound=0)

model += (4 * a + 5 * c + 2 * d + 2 * f <= 7200, "#1 constraint")
model += (4 * a + 3 * b + 4 * d + 4 * e + 2 * f <= 3600, "#2 constraint")
model += (3 * b + 2 * c + 5 * e + f <= 4800, "#3 constraint")
model += (5 * c + 4 * d + 2 * e + f <= 7200, "#4 constraint")
model += (b - c == 0, "#5 constraint")
model += (f - 2 * a == 0, "#6 constraint")

# Objective function
obj_func = 2.5 * a + b + 4 * c + 3 * d + 3.5 * e + 4 * f
model += obj_func

model

test-problem:
MAXIMIZE
2.5*A + 1*B + 4*C + 3*D + 3.5*E + 4*F + 0.0
SUBJECT TO
#1_constraint: 4 A + 5 C + 2 D + 2 F <= 7200

#2_constraint: 4 A + 3 B + 4 D + 4 E + 2 F <= 3600

#3_constraint: 3 B + 2 C + 5 E + F <= 4800

#4_constraint: 5 C + 4 D + 2 E + F <= 7200

#5_constraint: B - C = 0

#6_constraint: - 2 A + F = 0

VARIABLES
A Continuous
B Continuous
C Continuous
D Continuous
E Continuous
F Continuous

### Solve the problem

In [46]:
status = model.solve()

### Status

In [47]:
print(f"status: {model.status}, {LpStatus[model.status]}")

status: 1, Optimal


### Objective value

In [73]:
print(f"objective: {model.objective.value()}")

objective: 5700.000015


### Constraint values

In [49]:
for name, constraint in model.constraints.items():  print(f"{name}: {constraint.value()}")

#1_constraint: -1764.7058799999993
#2_constraint: -2.8421709430404007e-13
#3_constraint: 9.99999983264388e-06
#4_constraint: -2399.99999
#5_constraint: 0.0
#6_constraint: 1.0000000003174137e-05


### Model variable and their values

In [50]:
model.variables()

[A, B, C, D, E, F]

In [51]:
print(a.value())
print(b.value())
print(c.value())
print(d.value())
print(e.value())
print(f.value())

105.88235
917.64706
917.64706
0.0
0.0
211.76471


### Machine utilization check

In [67]:
## 1 constraint
print(4 * a.value() + 5 * c.value() + 2 * d.value() + 2 * f.value())
print(7200 - (4 * a.value() + 5 * c.value() + 2 * d.value() + 2 * f.value()))

5435.2941200000005
1764.7058799999995


In [68]:
## 2 constraint
print(4 * a.value() + 3 * b.value() + 4 * d.value() + 4 * e.value() + 2 * f.value())
print(3600 - (4 * a.value() + 3 * b.value() + 4 * d.value() + 4 * e.value() + 2 * f.value()))

3599.9999999999995
4.547473508864641e-13


In [69]:
## 3 constraint
print(3 * b.value() + 2 * c.value() + 5 * e.value() + f.value())
print(4800 - (3 * b.value() + 2 * c.value() + 5 * e.value() + f.value()))

4800.000010000001
-1.0000000656873453e-05


In [70]:
## 4 constraint
print(5 * c.value() + f.value())
print(7200 - (5 * c.value() + f.value()))

4800.000010000001
2399.9999899999993


In [63]:
## 5 constraint
print(b.value() - c.value())

0.0


In [65]:
## 6 constraint
print(f.value() - 2 * a.value())

1.0000000003174137e-05


## Report

1. The number of toys to manufacture: 

```A + B + C + D + E + F = 2152.94118 ~= 2152 (taking floor)```, where number of toys of:

A = 105.88235

B = 917.64706

C = 917.64706

D = 0.0

E = 0.0

F = 211.76471

2. The expected income: 

5700.000015 USD

Income separately from each toy:

A = 105.88235 toys * 2.5 USD = 264.705875 USD
 
B = 917.64706 toys * 1 USD = 917.64706 USD

C = 917.64706 toys * 4 USD = 3670.58824 USD

D = 0.0 toys * 3 USD = 0.0 USD 

E = 0.0 toys * 3.5 USD = 0.0 USD

F = 211.76471 toys * 4 USD = 847.05884 USD

3. The production time required on each machine (min):


(Machine 1) 4A + 0B + 5C + 2D + 0E + 2F = 5435.2941200000005

(Machine 2) 4A + 3B + 0C + 4D + 4E + 2F = 3599.9999999999995

(Machine 3) 0A + 3B + 2C + 0D + 5E + 1F = 4800.000010000001

(Machine 4) 0A + 0B + 5C + 4D + 2E + 1F = 4800.000010000001

## Questions

1. Which toy(s) production should be focused on?

The most income comes from toy C (3670.58824 USD).

2. Is there a toy that can be excluded for consideration for production?

The ones that do not provide any income (0.0 USD), i.e.:

- Toy D;

- Toy E.

3. Is there a machine that is not fully utilized?

Yes, the ones that resulted in negative constraint values, i.e.:

#1_constraint: -1764.7058799999993 

#2_constraint: -2.8421709430404007e-13

#4_constraint: -2399.99999

Since it means that its production time limit could be reduced by these values.

Note: Technically, #2_constraint is almost equal to zero which could be caused by minor calculation error. In case of rounding, it would be considered fully utilized.